In [16]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
import csv

#import training data
with open('training_data.txt','r') as train_f:
    train_iter = csv.reader(train_f, delimiter='|')
    trainlist = [i for i in train_iter]
train_raw = np.asarray(trainlist)
Label = train_raw[0,:]
x_train = train_raw[1:,0:-1]
y_train = train_raw[1:,-1]
x_train = np.asarray(x_train, dtype='float64')
y_train = np.asarray(y_train, dtype='int')

#import testing data
with open('testing_data.txt','r') as test_f:
    test_iter = csv.reader(test_f, delimiter='|')
    testlist = [i for i in test_iter]
x_test = np.asarray(testlist[1:])
x_test = np.asarray(x_test, dtype='float64')

In [30]:
from sklearn import preprocessing
x_train = preprocessing.normalize(x_train, norm='l2')
x_test = preprocessing.normalize(x_test, norm='l2')
x_train = preprocessing.scale(x_train)
x_test = preprocessing.scale(x_test)

In [18]:
'''
#feature selection: tfidf
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(x_train)
weight = transformer.idf_
select = np.argsort(weight)[500:]
x_train_select = x_train[:,select]
y_train_select = y_train
x_test_select = x_test[:,select]
'''

'\n#feature selection: chi-square\nfrom sklearn.feature_selection import chi2\n'

In [19]:
#split train and validation data
#rowindex = [i for i in np.random.randint(0,x_train_select.shape[0], size=3000)]
x_train_select_train = x_train_select[0:x_train_select.shape[0]/2,:]
y_train_select_train = y_train_select[0:x_train_select.shape[0]/2]
y_train_select_train[y_train_select_train==0] = -1
x_train_select_valid = x_train_select[x_train_select.shape[0]/2:,:]
y_train_select_valid = y_train_select[x_train_select.shape[0]/2:]
y_train_select_valid[y_train_select_valid==0] = -1

In [20]:

H = []    # store candidates

#svm candidates
print 'Start training SVM candidates...'
from sklearn import svm
c = [float(10**x) for x in range(-7, 4)]
Gamma = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1., 2.]
for i in range(len(c)):
    for j in range(len(Gamma)):
        clf = svm.SVC(C = c[i], gamma = Gamma[j], kernel='rbf')
        clf.fit(x_train_select_train, y_train_select_train)
        H.append(clf)
print 'SVM Candidates training Done\n'

print 'Start training Naive Bayes...'
#Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(x_train_select_train, y_train_select_train)
H.append(clf)

#BernoulliNB
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(x_train_select_train, y_train_select_train)
H.append(clf)

#MultinomialNB
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_select_train, y_train_select_train)
H.append(clf)
print 'Naive Bayes training Done\n'

print 'Start training Decision Tree...'
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
Max_depth=[1, 2, 3, 4, 5, 7, 10, 20]
N_estimators=[int(1.5**x) for x in range(1,15)]
for i in range(len(Max_depth)):
    clf = tree.DecisionTreeClassifier(max_depth=Max_depth[i])
    for j in range(len(N_estimators)):
        dlf = AdaBoostClassifier(base_estimator=clf, n_estimators=N_estimators[j])
        dlf.fit(x_train_select_train, y_train_select_train)
        H.append(dlf)
print 'Decision Tree training Down\n'


print 'Start training SVM candidates with kernel poly...'
Degree = [2, 3, 4, 5, 6]
for i in range(len(c)):
    for j in range(len(Degree)):
        clf = svm.SVC(C = c[i], kernel='poly', degree=Degree[j])
        clf.fit(x_train_select_train, y_train_select_train)
        H.append(clf)
print 'SVM Candidates training Done\n'

print 'Start training RandomForest candidates...'
#RandomForest candidates
from sklearn.ensemble import RandomForestClassifier
N_estimators = [100, 500, 1000]
Max_depth = [i+1 for i in range(10)]
Max_features = [int(1.5**x * np.sqrt(1000)) for x in range(-5,7)]
for i in range(len(N_estimators)):
    for j in range(len(Max_depth)):
        for k in range(len(Max_features)):
            clf = RandomForestClassifier(n_estimators=N_estimators[i], max_depth=Max_depth[j], max_features=Max_features[k])
            clf.fit(x_train_select_train, y_train_select_train)
            H.append(clf)
print 'RandomForest candidates training Done\n'

print 'Starting training KNN...'
#KNN
from sklearn.neighbors import KNeighborsClassifier
N_neighbors = [i+5 for i in range(10)]
P = [1, 2]
for i in range(len(N_neighbors)):
    for j in range(len(P)):
        clf = KNeighborsClassifier(n_neighbors=N_neighbors[i], p=P[j])
        clf.fit(x_train_select_train, y_train_select_train)
        H.append(clf)
print 'KNN training Down\n'

print 'Start training Logistic Regression\n'
from sklearn.linear_model import LogisticRegression
Penalty = ['l1', 'l2']
c = [float(10**x) for x in range(-7, 4)]
for i in range(len(c)):
    for j in range(len(Penalty)):
        clf = LogisticRegression(penalty=Penalty[j], C=c[i])
        clf.fit(x_train_select_train, y_train_select_train)
        H.append(clf)
print 'Logistic Regression training Down\n'

print 'Start Ensemble Selection...'
result = H[0].predict(x_train_select_valid)
score = sum(np.sign(result)==y_train_select_valid)*1.0/y_train_select_valid.shape[0]
y_test = H[0].predict(x_test_select)
model_number = 0
for i in range(1, len(H)):
    new = H[i].predict(x_train_select_valid)
    y = np.sign(new + result)
    if( (sum(y==y_train_select_valid)*1.0/y_train_select_valid.shape[0]) >= score ):
        result = new + result
        score = sum(y==y_train_select_valid)*1.0/y_train_select_valid.shape[0]
        y_test = H[i].predict(x_test_select) + y_test
        model_number += 1
print 'Ensemble Selection Done\n'



print result
print score
print y_test


Start training SVM candidates...
SVM Candidates training Done

Start training Naive Bayes...
Naive Bayes training Done

Start training Decision Tree...
Decision Tree training Down

Start training SVM candidates with kernel poly...
SVM Candidates training Done

Start training RandomForest candidates...
RandomForest candidates training Done

Starting training KNN...
KNN training Down

Start training Logistic Regression

Logistic Regression training Down

Start Ensemble Selection...
Ensemble Selection Done

[139  75  85 ..., 155 155 127]
0.566021867115
[123 163 103 ..., 163 163  73]


In [28]:
y_test = np.sign(y_test)

In [29]:
y_test2 = np.zeros(x_test.shape[0])
y_test2[y_test==1] = 1
y_test2[y_test==0] = 1
y_test2[y_test==-1] = 0
y = np.zeros((x_test.shape[0],2))
y[:,0] = [i+1 for i in range(x_test.shape[0])]
y[:,1] = y_test2
y = np.asarray(y, dtype='int')
with open('solution.txt','w') as output:
    a = csv.writer(output, delimiter=',')
    a.writerows(y)